In [1]:
import pandas as pd
import numpy as np
import os
import ast

In [2]:
rubrics = ['public_rubric.xls', 'private_rubric.xls']
target = 'assignment_0'

In [3]:
os.system(f'ipython nbconvert --to python {target}.ipynb');
exec(f'import {target} as submission')

In [11]:
def convert_to_script(target):
    script_already_exists = os.path.exists(target + '.py')
    if script_already_exists:
        os.renames(target + '.py', target + '.py.BACKUP')

    os.system(f'ipython nbconvert --to python {target}.ipynb');
    
    exec(f'import {target} as submission')
    
    return submission, script_already_exists

In [12]:
sub, already_exists = convert_to_script('assignment_0')

In [ ]:
sub.

In [4]:
def from_str(s):
    try:
        if s is np.nan:
            return ''
    except:
        pass
    
    try:
        if s is None:
            return ''
    except:
        pass
    
    try:
        if len(s) == 0:
            return ''
    except:
        pass
    
    try:
        return ast.literal_eval(s)
    except:
        return s

In [5]:
def grade(x, rubric):    
    if type(rubric) == list:
        passed = []
        failed = []
        for r in rubric:
            next_passed, next_failed = grade(x, r)
            passed.extend(next_passed)
            failed.extend(next_failed)
        return passed, failed
    
    r = pd.read_excel(rubric, header=0)
    
    passed = []
    failed = []
    
    for i in range(r.shape[0]):
        next_test = {}
        if r.loc[i]['function'] == 'identity':
            next_test['cmd'] = f'x.{r.loc[i]['input']} == {from_str(r.loc[i]['solution'])}'
            next_test['target'] = True)
        else:
            next_test['cmd'] = f"x.{r.loc[i]['function']}({str(from_str(r.loc[i]['input']))})"
            next_test['target'] = from_str(r.loc[i]['solution'])
        next_test['points'] = float(r.loc[i]['points'])
        next_test['rubric'] = rubric
        
        #FIXME: this part is broken...
        try:
            next_test['response'] = eval(next_test['cmd'])            
        except:
            next_test['response'] = None        
        try:            
            if next_test['response'] == next_test['target']:
                passed.append(next_test)
            else:
                failed.append(next_test)
        except:
            try:
                if np.isnan(next_test['target']):
                    passed.append(next_test)
                else:
                    failed.append(next_test)
            except:
                failed.append(next_test)
    
    return passed, failed

In [6]:
def report(passed, failed):
    possible = 0
    earned = 0
    
    def summarize(description, cases):
        if len(cases) == 0:
            return f'No cases {description.lower()}.'
        
        summary = [f'The following test cases {description}:']
        for i, c in enumerate(cases):
            summary.append(f"{i+1}. Command: {c['cmd'][2:]}")
            summary.append(f"\t Observed output: {c['response']}")
            summary.append(f"\t Expected output: {c['target']}")
            summary.append(f"\t Points: {c['points']}")
            summary.append('')
    
        return '\n'.join(summary[:-1])
    
    for p in passed:
        earned += p['points']
        possible += p['points']
    
    for f in failed:
        possible += f['points']
    
    print(f'Total points earned: {earned}/{possible} = {100 * np.round(float(earned)/(float(possible)), decimals=4)}%')
    print('\nDetails:\n')
    print(summarize('PASSED', passed))
    print('\n')
    print(summarize('FAILED', failed))

In [7]:
passed, failed = grade(submission, rubrics)
report(passed, failed)

Total points earned: 17.5/26.75 = 65.42%

Details:

The following test cases PASSED:
1. Command: say_hello()
	 Observed output: hello
	 Expected output: hello
	 Points: 0.5

2. Command: sum_1_n(3)
	 Observed output: 6
	 Expected output: 6
	 Points: 1.0

3. Command: sum_1_n(100)
	 Observed output: 5050
	 Expected output: 5050
	 Points: 1.0

4. Command: sum_1_n(-10)
	 Observed output: 0
	 Expected output: 0
	 Points: 2.0

5. Command: sum_1_n(0.5)
	 Observed output: 0
	 Expected output: 0
	 Points: 1.0

6. Command: sum_1_n([1, 2, 3])
	 Observed output: [1, 3, 6]
	 Expected output: [1, 3, 6]
	 Points: 2.0

7. Command: sum_1_n_no_checks(3)
	 Observed output: 6
	 Expected output: 6
	 Points: 1.0

8. Command: sum_1_n_no_checks(100)
	 Observed output: 5050
	 Expected output: 5050
	 Points: 1.0

9. Command: sum_1_n_no_checks(-10)
	 Observed output: 0
	 Expected output: 0
	 Points: 2.0

10. Command: sum_1_n([-100, 30, 7, 50])
	 Observed output: [0, 465, 28, 1275]
	 Expected output: [0, 465, 28, 